In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import threading
import urllib.request
import errno
import time 
import os

스크래핑 할 식물 이름들 불러오기

In [5]:
f = open("식물 학명 리스트_다육,선인장.txt", "r", encoding="utf-8")
plant_list = f.read().splitlines()
print(plant_list)
f.close()

#스크래핑 할 이미지 수
num = 250

# image 폴더가 있는지 검사 후 없으면 생성
if not os.path.exists("./img"):
        os.mkdir("./img")

['Selenicereus anthonyanus', 'Sansevieria cylindrica', "Sansevieria trifasciata 'Moonshine'", "Senecio candicans 'Angel wings'", 'Hoya carnosa', 'Cleistocactus colademononis', 'Yucca recurvifolia', 'Kalanchoe blosfeldian', 'Myrtillocactus geometrizans', 'Rhipsalis trigona', 'Sansevieria trifasciata var. laurentii', 'Euphorbia hypogaea', 'Agave attenuata', 'Plectranthus hadiensis var. tomentosus', 'Xerosicyos danguyi', 'Senecio rowleyanus', 'Opuntia microdasys', 'Conophytum bilobum', 'Tillandsia Cyanea (Wallisia cyanea)', 'Euphorbia lactea ‘White Ghost’ Haw.', 'Beaucarnea recurvata (Nolina tuberculata)', 'Crassula ovata', 'Ceropegia woodii', 'Peperomia pepperspot', 'Haworthia obtusa', 'Haworthia fasciata', 'Peperomia prostrata', 'Euphorbia tirucalli', 'Syzygium australe', 'Aloe barbadens', 'Adenium obesum', "Aeonium arboreum 'Zwartkop'", 'Mammillaria gracilis cv. Arizona Snowcap', "Sansevieria 'Superba'", 'Opuntia monacantha', 'Euphorbia trigona', 'Aloe arborescens', "Sansevieria trifas

In [6]:
driver = webdriver.Chrome("/Users/PC/Downloads/chromedriver/chromedriver")

for plant in plant_list:
    #식물에 해당하는 폴더 생성
    if not (os.path.exists("./img/" + str(plant))):
        os.mkdir((os.path.join("./img/" + str(plant))))

    # 구글 이미지 검색 url
    driver.get("https://www.google.co.kr/imghp?hl=ko")
    driver.implicitly_wait(2)

    elem = driver.find_element_by_name("q") 

    #검색창에 검색할 식물 이름을 넣고 엔터
    elem.send_keys(str(plant))
    elem.send_keys(Keys.RETURN)
    time.sleep(1)
        
    #스크롤 내리기 (더보기 버튼 누르기)
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        #현재 다운로드 할 수 있는 이미지 개수
        imgs = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
        #print("로드된 이미지 개수 : ", len(imgs)) #-> 한번 더보기 클릭 시 100개 씩 로딩   

        if len(imgs) >= num:
            break

        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            try:
                driver.find_element_by_css_selector(".mye4qd").click()
            except:
                break

        last_height = new_height


    #이미지 수집 시작
    count = 1

    for img in imgs:
        try:
            driver.execute_script("arguments[0].click();", img)
            time.sleep(2)
            
            # 큰 이미지 주소 추출
            if count == 1:
                target = driver.find_elements_by_css_selector("img.n3VNCb")[0]
            else:
                target = driver.find_elements_by_css_selector("img.n3VNCb")[1]

            img_src = target.get_attribute('src') 
            
            #HTTP Error 403: Forbidden 에러 방지
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozila/5.0')]
            urllib.request.install_opener(opener)   
            
            if not os.path.exists("./img/" + str(plant) + "/" + str(count) + ".jpg"):
                urllib.request.urlretrieve(img_src , "./img/" + str(plant) + "/" + str(count) + ".jpg")
                count += 1
            else:
                count += 1
                continue
            
        except Exception as e:
            print("Error!" + str(count))
            print(e)
            pass
        
        if count == num + 1:
            print(str(plant) + "의 총 이미지 수 : ", count)
            break
            
driver.close()

print("다운로드 완료!")

C:\Users\PC\AppData\Local\Temp\ipykernel_9172\26642077.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/PC/Downloads/chromedriver/chromedriver")
C:\Users\PC\AppData\Local\Temp\ipykernel_9172\26642077.py:12: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  elem = driver.find_element_by_name("q")
C:\Users\PC\AppData\Local\Temp\ipykernel_9172\26642077.py:27: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  imgs = driver.find_elements_by_css_selector(".rg_i.Q4LuWd")
C:\Users\PC\AppData\Local\Temp\ipykernel_9172\26642077.py:54: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  target = driver.find_elements_by_css_selector("img.n3VNCb")[0]
C:\Users\PC\AppData\Local\Temp

Error!96
HTTP Error 503: Service Temporarily Unavailable
Error!126
HTTP Error 503: Service Temporarily Unavailable
Error!205
<urlopen error [SSL: DH_KEY_TOO_SMALL] dh key too small (_ssl.c:997)>
Error!210
HTTP Error 503: Service Temporarily Unavailable
Kalanchoe blosfeldian의 총 이미지 수 :  251
Myrtillocactus geometrizans의 총 이미지 수 :  251


C:\Users\PC\AppData\Local\Temp\ipykernel_9172\26642077.py:37: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector(".mye4qd").click()


Error!38
Message: no such window: window was already closed
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0041B8F3+2406643]
	Ordinal0 [0x003AAF31+1945393]
	Ordinal0 [0x0029C748+837448]
	Ordinal0 [0x0029144A+791626]
	Ordinal0 [0x002924B2+795826]
	Ordinal0 [0x0028BF09+769801]
	Ordinal0 [0x0029DAC0+842432]
	Ordinal0 [0x002F3E62+1195618]
	Ordinal0 [0x002E4096+1130646]
	Ordinal0 [0x002BE636+976438]
	Ordinal0 [0x002BF546+980294]
	GetHandleVerifier [0x00689612+2498066]
	GetHandleVerifier [0x0067C920+2445600]
	GetHandleVerifier [0x004B4F2A+579370]
	GetHandleVerifier [0x004B3D36+574774]
	Ordinal0 [0x003B1C0B+1973259]
	Ordinal0 [0x003B6688+1992328]
	Ordinal0 [0x003B6775+1992565]
	Ordinal0 [0x003BF8D1+2029777]
	BaseThreadInitThunk [0x74DCFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76ED7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x76ED7A4E+238]

Error!38
Message: no such window: window was already closed
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=101.0.4951.67)
Stacktrace:
Backtrace:
	Ordinal0 [0x0041B8F3+2406643]
	Ordinal0 [0x003AAF31+1945393]
	Ordinal0 [0x0029C748+837448]
	Ordinal0 [0x0029144A+791626]
	Ordinal0 [0x002924B2+795826]
	Ordinal0 [0x0028BF09+769801]
	Ordinal0 [0x0029DAC0+842432]
	Ordinal0 [0x002F3E62+1195618]
	Ordinal0 [0x002E4096+1130646]
	Ordinal0 [0x002BE636+976438]
	Ordinal0 [0x002BF546+980294]
	GetHandleVerifier [0x00689612+2498066]
	GetHandleVerifier [0x0067C920+2445600]
	GetHandleVerifier [0x004B4F2A+579370]
	GetHandleVerifier [0x004B3D36+574774]
	Ordinal0 [0x003B1C0B+1973259]
	Ordinal0 [0x003B6688+1992328]
	Ordinal0 [0x003B6775+1992565]
	Ordinal0 [0x003BF8D1+2029777]
	BaseThreadInitThunk [0x74DCFA29+25]
	RtlGetAppContainerNamedObjectPath [0x76ED7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x76ED7A4E+238]
